# Working with Twitter in Python

In this tutorial we will be using the [tweepy](http://www.tweepy.org/) API for twitter.

To install it run:
* pip install tweepy

Next we need to register with Twitter
* Go to [https://dev.twitter.com](https://dev.twitter.com)
* Click on "Manage Your Apps" and create a new App
* If your project doesn't have a website yet, just use http://example.com
* For this tutorial, we do not need an Oauth token since we just care about scraping Twitter data

Now we have a consumer API token, but we need secret tokens for our app
* Click on Keys and Access tokens
* Then click on the "Create my access token button"

# Create a token file (twitter.cfg)

Add in your keys so the file looks like:

[api]

consumer_key = %consumer_key%

consumer_secret = %consumer_secret%

access_token = %access_token%

access_token_secret = %access_token_secret%

In [1]:
from pprint import pprint
import tweepy
import configparser
import json

parser = configparser.ConfigParser()
parser.read(['twitter.cfg'])
consumer_key = parser.get('api', 'consumer_key')
consumer_secret = parser.get('api', 'consumer_secret')
access_token = parser.get('api', 'access_token')
access_token_secret = parser.get('api', 'access_token_secret')

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
# Getting Trending HashTags
# Based on https://woe.spum.org/ (e.g. search United States)
trends = api.trends_place(23424977)
pprint(trends)

[{u'as_of': u'2016-04-01T00:41:52Z',
  u'created_at': u'2016-04-01T00:40:31Z',
  u'locations': [{u'name': u'United States', u'woeid': 23424977}],
  u'trends': [{u'name': u'#BernieInTheBronx',
               u'promoted_content': None,
               u'query': u'%23BernieInTheBronx',
               u'tweet_volume': 33867,
               u'url': u'http://twitter.com/search?q=%23BernieInTheBronx'},
              {u'name': u'#ImSoSick',
               u'promoted_content': None,
               u'query': u'%23ImSoSick',
               u'tweet_volume': 29713,
               u'url': u'http://twitter.com/search?q=%23ImSoSick'},
              {u'name': u'#LegendsOfTomorrow',
               u'promoted_content': None,
               u'query': u'%23LegendsOfTomorrow',
               u'tweet_volume': None,
               u'url': u'http://twitter.com/search?q=%23LegendsOfTomorrow'},
              {u'name': u'#80sAnatomyMovies',
               u'promoted_content': None,
               u'query': u'%2380

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [4]:
trends_list = [i['name'] for i in trends[0]['trends']]
print trends_list

[u'#BernieInTheBronx', u'#ImSoSick', u'#LegendsOfTomorrow', u'#80sAnatomyMovies', u'#IdolTop3', u'April Fools', u'Saint Pablo', u'BURY TROPES NOT US', u'New Hope', u'Richmond', u"Cal O'Reilly", u'Christian Beadles', u'Rick Nash', u'Kessel', u'Pot Roast', u'Matt Barnes', u'Antonin Scalia School of Law', u'Miller Grove', u'James Neal', u'PETA', u'Daniel Thomas', u'Mirotic', u'The God Smile', u'#FanGirlProblems', u'#KCONLiveChat', u'#Miitomo', u'#OutbackMobileApp', u'#HopeOnItunes', u'#NovelWaysToMakeFriends', u'#educolor', u'#YouTubeBLACK', u'#LifeInPieces', u'#VogueCam', u'#alwx', u'#YMATA', u'#Axxess', u'#whatisschool', u'#Believeland', u'#GirlsWriteIn', u'#ASKJENN', u'#DIDforSAAM', u'#YourNextPC', u'#Greys', u'#TraderJoes', u'#STL100', u'#PeterPanMusicVideo', u'#TheMoonIsBlue', u'#DTLAsksAlessia', u'#clarity2016', u'#NYRvsCAR']


# Now let's set upstreaming!

In [5]:
from tweepy.streaming import StreamListener

In [6]:
class StdOutStreamer(StreamListener):
    def __init__(self):
        super(StdOutStreamer, self).__init__()
        self.tweet_cnt = 0
        self.retweet_cnt = 0

    def on_data(self, data):
        data = json.loads(data)
        # Check if there is text tied to the tweet
        if 'text' in data:
            print data['text']
            if data['text'].strip().startswith('RT'):
                self.retweet_cnt += 1
            else:
                self.tweet_cnt += 1
        # Just to exit
        if self.tweet_cnt >= 100:
            raise Exception("Too many tweets")
    def on_error(self, status):
        print status
        print "Tweet Cnt: %d, RT Cnt: %d"%(self.retweet_cnt, self.tweet_cnt)

In [8]:
try:
    l = StdOutStreamer()
    stream = tweepy.Stream(auth, l)
    stream.filter(track=trends_list)
except KeyboardInterrupt, e:
    pass
except Exception, e:
    pass

RT @NWSBirmingham: Here's our 00Z sounding from KBMX #alwx https://t.co/3OSqAfP1Z9
RT @versacenil: ok gn twitter i will be listening to Saint Pablo and crying for the rest of the night. txt me if you need me.
RT @Dory: cute april fools day prank: transfer $1,000,000 into my bank account
Zora could've been the new Cephalo. Instead the rapist lives &amp; becomes a hero &amp; the lesbian dies. BURY TROPES NOT US https://t.co/xqe4qt5IFN
#KCONLIVECHAT IF U DONT BRING WINNER IM GONNA CRY LOL
RT @angelofmusic27: "Okay, I have one son." 😂😂😂 #LifeInPieces
RT @AmericanIdol: RETWEET for some #IdolLove! We're following a few lucky fans right now. 🎤🎶❤️ #IdolTop3
RT @AmericanIdol: RETWEET if you're watching the #IdolTop3 reveal! We are LIVE right now. https://t.co/ywiXw3ZiyQ
Bernie is our only hope, Hillary has already sold herself to private corporations. #BernieInTheBronx
RT @cnnbrk: The trooper injured during Richmond, Virginia, bus station shooting has died, a state police official says. https:/

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
